# Import Modules

In [1]:
import os
import pandas as pd
from credentials import *
import snowflake.connector
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine, text

c:\Users\ASANTE\miniconda3\envs\SNOWFLAKE\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (12.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


# Helper Functions

In [2]:
def make_conn(**db_parameters):
    """
    Composes a SQLAlchemy connect string from the given database connection parameters.

    https://github.com/snowflakedb/snowflake-sqlalchemy#escaping-special-characters-such-as---signs-in-passwords
    
    """
    try:
        engine = create_engine(URL(**db_parameters))
        return engine.connect()
    
    except snowflake.connector.errors.ProgrammingError as e:
        return f"Error: {e}"
    
    finally:
        engine.connect().close()
        engine.dispose()

In [3]:
conn = make_conn(role=ROLE, account=ACCOUNT_ID, user=USERNAME, password=PASSWORD, database='', schema='', warehouse='')